In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [2]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "Intent_train_data.csv")
df = pd.read_csv(train_file, delimiter=',', header=None)
df = df[[0, 1, 2]]

In [3]:
df.columns = ['query', 'intent', 'intent_info']
seq_len = list(map(lambda x : len(x.split(' ')), df['query']))
max(seq_len)  # 문장 최대길이

8

In [4]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = max(seq_len)

def GlobalParams():
    global MAX_SEQ_LEN

In [5]:
# '질문(query)' 과 '의도(intent)'
queries = df['query'].tolist()
intents = df['intent'].tolist()

In [6]:
queries[:20]

['사이드메뉴',
 '사이드 메뉴',
 '빵종류',
 '종류',
 '음료종류',
 '커피종류',
 '식사종류',
 '에이드종류',
 '사이드메뉴?',
 '사이드 메뉴?',
 '빵종류?',
 '종류?',
 '음료종류?',
 '커피종류?',
 '식사종류?',
 '에이드종류?',
 '사이드메뉴는 뭐 있어?',
 '사이드 메뉴는 뭐 있어?',
 '빵종류는 뭐 있어?',
 '종류는 뭐 있어?']

In [7]:
intents[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# 전처리 모듈 불러오기
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [9]:
# 단어 시퀀스 생성 (가장 시간 많이 걸림)
# 해당 단어에 매칭되는 번호로 시퀀스 생성

# ★ 시간 제법 걸림 * 몇십초 정도..

sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [10]:
sequences

[[7837, 67],
 [7837, 67],
 [1],
 [8941],
 [1],
 [1],
 [1],
 [1, 8941],
 [7837, 67],
 [7837, 67],
 [1],
 [8941],
 [1],
 [1],
 [1],
 [1, 8941],
 [7837, 67, 552, 25],
 [7837, 67, 552, 25],
 [1, 552, 25],
 [8941, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 8941, 552, 25],
 [7837, 67],
 [7837, 67],
 [1],
 [8941],
 [1],
 [1],
 [1],
 [1, 8941],
 [7837, 67, 42, 21],
 [7837, 67, 42, 21],
 [1, 42, 21],
 [8941, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 8941, 42, 21],
 [7837, 67, 42, 21],
 [7837, 67, 42, 21],
 [1, 42, 21],
 [8941, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 42, 21],
 [1, 8941, 42, 21],
 [7837, 67, 25],
 [7837, 67, 25],
 [1, 25],
 [8941, 25],
 [1, 25],
 [1, 25],
 [1, 25],
 [1, 8941, 25],
 [7837, 67, 25],
 [7837, 67, 25],
 [1, 25],
 [8941, 25],
 [1, 25],
 [1, 25],
 [1, 25],
 [1, 8941, 25],
 [7837, 67, 552, 25],
 [7837, 67, 552, 25],
 [1, 552, 25],
 [8941, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 552, 25],
 [1, 8941, 552, 25],
 [7837, 67, 627, 8],
 [78

In [11]:
# ② 단어 인덱스 시퀀스 벡터 
# 단어 시퀀스 벡터 크기 (MAX_SEQ_LEN 로 동일하게 맞추기, 패딩처리)
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [12]:
padded_seqs

array([[ 7837,    67,     0, ...,     0,     0,     0],
       [ 7837,    67,     0, ...,     0,     0,     0],
       [    1,     0,     0, ...,     0,     0,     0],
       ...,
       [ 3232,  4035,     0, ...,     0,     0,     0],
       [10258,   188,     0, ...,     0,     0,     0],
       [ 3599,  3566,     0, ...,     0,     0,     0]])

In [13]:
len(p.word_index)

17751

In [137]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(32)
val_ds = ds.skip(train_size).take(val_size).batch(32)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(32)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 30
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

## train_ds

In [138]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(12, name='logits')(dropout_hidden)  # 최종적으로 12가의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(12, activation=tf.nn.softmax)(logits)


In [139]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [140]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 10, 128)      2272256     ['input_2[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 10, 128)      0           ['embedding_1[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 8, 128)       49280       ['dropout_2[0][0]']              
                                                                                            

In [ ]:
# 모델학습
# ★ 시간 걸림 ★ 
checkpoint_cb = keras.callbacks.ModelCheckpoint('./models/intent/intent_model.h5', save_best_only=True) # 제일 좋은 모델 저장
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1,
         callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/30
317/317 [==============================] - 12s 35ms/step - loss: 0.9273 - accuracy: 0.7066 - val_loss: 0.1899 - val_accuracy: 0.9227
Epoch 2/30
317/317 [==============================] - 11s 34ms/step - loss: 0.2432 - accuracy: 0.9138 - val_loss: 0.1476 - val_accuracy: 0.9344
Epoch 3/30
317/317 [==============================] - 11s 33ms/step - loss: 0.1915 - accuracy: 0.9258 - val_loss: 0.1467 - val_accuracy: 0.9323
Epoch 4/30
317/317 [==============================] - 11s 33ms/step - loss: 0.1772 - accuracy: 0.9294 - val_loss: 0.1450 - val_accuracy: 0.9372
Epoch 5/30
317/317 [==============================] - 11s 34ms/step - loss: 0.1656 - accuracy: 0.9297 - val_loss: 0.1295 - val_accuracy: 0.9392
Epoch 6/30
317/317 [==============================] - 10s 33ms/step - loss: 0.1591 - accuracy: 0.9315 - val_loss: 0.1418 - val_accuracy: 0.9261
Epoch 7/30
317/317 [==============================] - 10s 33ms/step - loss: 0.1592 - accuracy: 0.9311 - val_loss: 0.1269 - val_accuracy:

In [ ]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

## 의도분류 모듈 테스트

In [ ]:
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [ ]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [ ]:
query = "개샛꺄"

In [ ]:
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

In [ ]:
print(query)
print('의도 예측 클래스 : ', predict)
print('의도 예측 레이블 : ', predict_label)